In [71]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [72]:
#Importing the required libraries to read,visualize and model the givn dataset files
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#import plotly.graph_objects as go
#import plotly.express as px
#import missingno as msno 
import warnings
warnings.filterwarnings("ignore")
import re
import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
import nltk
from nltk.tokenize import word_tokenize,RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from platform import python_version
print (python_version())

3.8.3


[nltk_data] Downloading package stopwords to /Users/rohit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<b>1. Read CSV Files

In [73]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [74]:
train.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [75]:
train[0:5]

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [76]:
train.shape

(7613, 5)

<b>2. Store a list with non-null keywords

In [77]:
keyword_list = train[train.keyword.notnull()]
keyword_list

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
...,...,...,...,...,...
7578,10830,wrecked,NaN,@jt_ruff23 @cameronhacker and I wrecked you both,0
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0
7581,10833,wrecked,Lincoln,@engineshed Great atmosphere at the British Li...,0


<b>3. Store another list with NULL Keywords

In [78]:
null_keyword = train[train.keyword.isnull()]
null_keyword

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


<b>4. Find unique keywords

In [79]:
unique_keyword = train.keyword.unique()
unique_keyword = unique_keyword.tolist()
unique_keyword.remove(np.nan)
unique_keyword

['ablaze',
 'accident',
 'aftershock',
 'airplane%20accident',
 'ambulance',
 'annihilated',
 'annihilation',
 'apocalypse',
 'armageddon',
 'army',
 'arson',
 'arsonist',
 'attack',
 'attacked',
 'avalanche',
 'battle',
 'bioterror',
 'bioterrorism',
 'blaze',
 'blazing',
 'bleeding',
 'blew%20up',
 'blight',
 'blizzard',
 'blood',
 'bloody',
 'blown%20up',
 'body%20bag',
 'body%20bagging',
 'body%20bags',
 'bomb',
 'bombed',
 'bombing',
 'bridge%20collapse',
 'buildings%20burning',
 'buildings%20on%20fire',
 'burned',
 'burning',
 'burning%20buildings',
 'bush%20fires',
 'casualties',
 'casualty',
 'catastrophe',
 'catastrophic',
 'chemical%20emergency',
 'cliff%20fall',
 'collapse',
 'collapsed',
 'collide',
 'collided',
 'collision',
 'crash',
 'crashed',
 'crush',
 'crushed',
 'curfew',
 'cyclone',
 'damage',
 'danger',
 'dead',
 'death',
 'deaths',
 'debris',
 'deluge',
 'deluged',
 'demolish',
 'demolished',
 'demolition',
 'derail',
 'derailed',
 'derailment',
 'desolate',
 'de

In [80]:
b = train[train.location.notnull()]
b.shape

(5080, 5)

<b>5. Fill in NULL Keywords list with Keywords based on Unique Keywords Occurances in Text Data

In [81]:
for i, j in enumerate(unique_keyword):
    for k, txt in enumerate(null_keyword['text']):
        if j in txt:
            null_keyword.keyword[k] = j

In [82]:
null_keyword.dtypes

id           int64
keyword     object
location    object
text        object
target       int64
dtype: object

<b>6. Concat NULL and existing keyword lists

In [83]:
train2 = pd.concat([keyword_list, null_keyword])

In [84]:
train2[train2['keyword']=='collapse']

,id,keyword,location,text,target
1596,2304,collapse,NaN,Runaway Minion Causes Traffic Collapse in Dubl...,1
1597,2306,collapse,"Highland Park, CA",Time collapse is such a cool video technique. ...,0
1598,2307,collapse,Scotland,Would a paramedic really do that? Leave someon...,1
1599,2308,collapse,Swan River,@GeoffRickly I don't see the option to buy the...,0
1600,2309,collapse,"Kolkata, India",Warne Ponting shocked by Australian collapse -...,0
1601,2311,collapse,Melrose,Watch The Terrifying Moment Two Giant Cranes C...,1
1602,2312,collapse,"Jubail IC, Saudi Arabia",@BasilDudin The 'barbaric Saudies' as you said...,0
1603,2314,collapse,New York City,Interview on The Collapse of Materialism Best ...,1
1604,2315,collapse,NaN,Economic Collapse Investing: Specific actions ...,1
1605,2317,collapse,"Los Angeles, CA",What would you do if you were trapped in a col...,0


<b>7. Define Function for:-<br>
    a. Converting to Lower Case<br>
    b. Tokenizing<br>
    c. Removing Punctuations<br>
    d. Removing Stopwords<br>
    e. Removing Links<br>

In [85]:
def clean_data(text):
    app_text =[]
    for i in text:
        i = i.lower()
        splut = i.split()
        #tok = nltk.tokenize.RegexpTokenizer(r'\w+')
        table = str.maketrans("","",string.punctuation)
        clean_punc = [w.translate(table) for w in splut]
        #print(clean_punc)
        remove_sw = [s for s in clean_punc if s not in stopwords.words('english')]
        remove_links = [x for x in remove_sw if not (x.startswith('http') | x.startswith('www'))]
        collab = ' '.join(remove_links)  
        app_text.append(collab)       
    return app_text

<b>8. Apply function to training data

In [86]:
train2C = train2.copy()
train2C['text'] = clean_data(train2C['text'])
train2C['text'].head()

31                      bbcmtd wholesale markets ablaze
32                      always try bring heavy metal rt
33    africanbaze breaking newsnigeria flag set abla...
34                                    crying set ablaze
35                 plus side look sky last night ablaze
Name: text, dtype: object

<b>9. Apply function to testing data

In [87]:
test2C = test.copy()
test2C['text'] = clean_data(test2C['text'])
test2C.head()

,id,keyword,location,text
0,0,NaN,NaN,happened terrible car crash
1,2,NaN,NaN,heard earthquake different cities stay safe ev...
2,3,NaN,NaN,forest fire spot pond geese fleeing across str...
3,9,NaN,NaN,apocalypse lighting spokane wildfires
4,11,NaN,NaN,typhoon soudelor kills 28 china taiwan


In [88]:
train2C

,id,keyword,location,text,target
31,48,ablaze,Birmingham,bbcmtd wholesale markets ablaze,1
32,49,ablaze,Est. September 2012 - Bristol,always try bring heavy metal rt,0
33,50,ablaze,AFRICA,africanbaze breaking newsnigeria flag set abla...,1
34,52,ablaze,"Philadelphia, PA",crying set ablaze,0
35,53,ablaze,"London, UK",plus side look sky last night ablaze,0
...,...,...,...,...,...
7608,10869,NaN,NaN,two giant cranes holding bridge collapse nearb...,1
7609,10870,NaN,NaN,ariaahrary thetawniest control wild fires cali...,1
7610,10871,NaN,NaN,m194 0104 utc5km volcano hawaii,1
7611,10872,NaN,NaN,police investigating ebike collided car little...,1


In [89]:
for i, j in enumerate(unique_keyword):
    for k in range(len(train2C)):
        if(train2C.loc[k,'keyword'] == np.nan):
            if(j in train2C.loc[k,'text']):
                train2C.loc[k, 'keyword'] = j

In [90]:
train3C = train2.copy()
train3C

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [38]:
import re
for i, j in enumerate(train3C['text']):
    train3C['text'][i] = re.sub(r'[^\w\s]','',j)
train3C

,id,keyword,location,text,target
31,48,ablaze,Birmingham,Rene Ablaze amp Jacinta Secret 2k13 Fallen Sk...,1
32,49,ablaze,Est. September 2012 - Bristol,Navista7 Steve these fires out here are someth...,0
33,50,ablaze,AFRICA,NowPlaying Rene Ablaze amp Ian Buff Magnitude...,1
34,52,ablaze,"Philadelphia, PA",nxwestmidlands huge fire at Wholesale markets ...,0
35,53,ablaze,"London, UK",ablaze what time does your talk go until I don...,0
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,aria_ahrary TheTawniest The out of control wil...,1
7610,10871,NaN,NaN,M194 0104 UTC5km S of Volcano Hawaii httptcozD...,1
7611,10872,NaN,NaN,Police investigating after an ebike collided w...,1


In [91]:
train2C.head()

,id,keyword,location,text,target
31,48,ablaze,Birmingham,bbcmtd wholesale markets ablaze,1
32,49,ablaze,Est. September 2012 - Bristol,always try bring heavy metal rt,0
33,50,ablaze,AFRICA,africanbaze breaking newsnigeria flag set abla...,1
34,52,ablaze,"Philadelphia, PA",crying set ablaze,0
35,53,ablaze,"London, UK",plus side look sky last night ablaze,0


<h1> Pre-processing with TF-IDF

In [99]:
tfidf = TfidfVectorizer(min_df = 2, max_df = 0.5, ngram_range = (1,2))
train_tf = tfidf.fit_transform(train2C['text'])
test_tf = tfidf.transform(test2C['text'])

In [100]:
train_tf

<7613x11672 sparse matrix of type '<class 'numpy.float64'>'
	with 77973 stored elements in Compressed Sparse Row format>

In [101]:
x_train_tf, x_test_tf, y_train_tf, y_test_tf = train_test_split(train_tf, train2C.target, test_size = 0.2, random_state=17)

In [102]:
def fit_predict(model, x_train, x_test, y_train, y_test):
    clf = model
    clf.fit(x_train, y_train)
    y_hat = clf.predict(x_test)
    confusion_matrix(y_test, y_hat)
    print(classification_report(y_test, y_hat))
    print('-'*100)
    print("{}".format(model))
    print('-'*100)
    print('Training Accuracy: {}%'.format(round(clf.score(x_train, y_train)*100)))
    print('-'*100)
    print('Test Data Accuracy: {}%'.format(round(accuracy_score(y_test,y_hat)*100)))
    print('-'*100) 

In [103]:
from sklearn.linear_model import LinearRegression
models=[LogisticRegression(C=1.0), SVC(), DecisionTreeClassifier(), RandomForestClassifier(), KNeighborsClassifier(n_neighbors=5)]

In [104]:
for model in models:
    fit_predict(model, x_train_tf, x_test_tf, y_train_tf, y_test_tf)

              precision    recall  f1-score   support

           0       0.79      0.91      0.85       880
           1       0.84      0.67      0.75       643

    accuracy                           0.81      1523
   macro avg       0.82      0.79      0.80      1523
weighted avg       0.81      0.81      0.80      1523

----------------------------------------------------------------------------------------------------
LogisticRegression()
----------------------------------------------------------------------------------------------------
Training Accuracy: 89%
----------------------------------------------------------------------------------------------------
Test Data Accuracy: 81%
----------------------------------------------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.79      0.91      0.84       880
           1       0.85      0.66      0.74       643

    accuracy                         

In [107]:
clf_final = LogisticRegression(C=1.0)
clf_final.fit(train_tf, train2C.target)
y_hat_final = clf_final.predict(test_tf)

In [108]:
y_hat_final

array([1, 1, 1, ..., 1, 1, 1])

In [112]:
sub = pd.DataFrame()
sub['id'] = test['id']
sub['target'] = y_hat_final
sub.to_csv('submission.csv',index=False)